In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from tqdm import trange
import pandas as pd

# Selenium

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')

s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options = options,service = s)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/home/cosbi/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache


In [3]:
driver.get("https://mops.twse.com.tw/mops/web/t100sb02_1")

In [4]:
select_element = driver.find_element(by = By.NAME, value = "month")
select_object = Select(select_element)
select_object.select_by_index(1)

In [5]:
driver.find_elements(by = By.XPATH, value = "//input[@type='button']")[1].click()

In [7]:
driver.find_elements(by = By.TAG_NAME, value = "button")[0].click()

# Bs4

In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [7]:
Id, name, date, time, place, message, ch_file, en_file, more_information, video_address, attention = [[] for x in range(11)]

In [8]:
result_even = soup.find_all("tr", class_ = "even")
result_odd = soup.find_all("tr", class_ = "odd")
result_total = result_even + result_odd

for i in trange(len(result_total)):
    data_td = result_total[i].find_all("td")

    for ptr in range(len(data_td)):
        if ptr == 0:
            Id.append(data_td[ptr].getText())
        elif ptr == 1:
            name.append(data_td[ptr].getText())
        elif ptr == 2:
            temp = data_td[ptr].getText().replace("/", "-")
            newYear = str(int(temp.split("-")[0]) + 1911)
            temp = temp.replace(temp.split("-")[0], "")
            temp = newYear + temp
            date.append(temp)
        elif ptr == 3:
            time.append(data_td[ptr].getText())
        elif ptr == 4:
            temp = data_td[ptr].getText()
            # temp = temp.replace("/r", "")
            # temp = temp.replace("/n", "")

            place.append(temp)
        elif ptr == 5:
            temp = data_td[ptr].getText()
            # temp = temp.replace("/r", "")
            # temp = temp.replace("/n", "")

            message.append(temp)
        elif ptr == 6:
            temp = data_td[ptr].getText()

    #         if ".pdf" in temp:
    #             download_payload = {
    #                 "step": "9",
    #                 "filePath": "/home/html/nas/STR/",
    #                 "fileName": str(temp),
    #                 "functionName": "t100sb02_1"
    #             }

    #             while(True):
    #                 try:
    #                     download_response = requests.post("https://mops.twse.com.tw/server-java/FileDownLoad"
    #                                          , data = download_payload)
    #                 except:
    #                     logging.error('Ch file download error')

    #                 data = download_response.content
    #                 tm.sleep(1)

    #                 if(len(data) != 496):
    #                     break

    #             if not os.path.isdir(rootPath + "file"):
    #                 os.mkdir(rootPath + "file")

    #             if not os.path.isdir(rootPath + "file/" + dic_name):
    #                     os.mkdir(rootPath + "file/" + dic_name)

    #             if not os.path.isdir(rootPath + "file/" + dic_name + "/ch"):
    #                 os.mkdir(rootPath + "file/" + dic_name + "/ch")

    #             with open(rootPath + "file/" + dic_name + "/ch/" + Id[-1] + "_" + temp, 'wb') as s:
    #                 s.write(data)

    #             ch_file.append(Id[-1] + "_" + temp)
    #         else:
    #             ch_file.append("null")
        elif ptr == 7:
            temp = data_td[ptr].getText()

    #         if ".pdf" in temp:
    #             download_payload = {
    #                 "step": "9",
    #                 "filePath": "/home/html/nas/STR/",
    #                 "fileName": str(temp),
    #                 "functionName": "t100sb02_1"
    #             }

    #             while(True):
    #                 try:
    #                     download_response = requests.post("https://mops.twse.com.tw/server-java/FileDownLoad"
    #                                          , data = download_payload)
    #                 except:
    #                     logging.error('En file download error')

    #                 data = download_response.content
    #                 tm.sleep(1)

    #                 if(len(data) != 496):
    #                     break

    #             if not os.path.isdir(rootPath + "file"):
    #                 os.mkdir(rootPath + "file")

    #             if not os.path.isdir(rootPath + "file/" + dic_name):
    #                 os.mkdir(rootPath + "file/" + dic_name)

    #             if not os.path.isdir(rootPath + "file/" + dic_name + "/en"):
    #                 os.mkdir(rootPath + "file/" + dic_name + "/en")

    #             with open(rootPath + "file/" + dic_name + "/en/" + Id[-1] + "_" + temp, 'wb') as s:
    #                 s.write(data)

    #             en_file.append(Id[-1] + "_" + temp)
    #         else:
    #             en_file.append("null")
        elif ptr == 8:
            more_information.append(data_td[ptr].getText())
        elif ptr == 9:
            video_address.append(data_td[ptr].getText())
        elif ptr == 10:
            if(data_td[ptr].getText() == ""):
                attention.append("無")
            else:
                attention.append(data_td[ptr].getText())

100%|██████████| 66/66 [00:00<00:00, 13243.27it/s]


In [9]:
df = pd.DataFrame({"Co_ID" : Id, "Co_name" : name, "Date" : date, "Time" : time, "Form" : place, "Message" : message
                      , "More information" : more_information, "Video address" : video_address, "Attention" : attention})

In [10]:
df

,Co_ID,Co_name,Date,Time,Form,Message,More information,Video address,Attention
0,2454,聯發科,2022-01-27,15:00,線上法說會,(1) 公布本公司2021年第4季財務報告及2022年第1季業績展望。 (2) 參加方式：請...,https://www.mediatek.com/,\n影音資訊網址：\nhttps://webpage-ott2b.cdn.hinet.net...,無
1,2408,南亞科,2022-01-26,15:00,線上法說會,2021年第4季營運狀況說明,https://www.nanya.com,\n影音資訊網址：\nhttps://www.nanya.com/tw/IR/13/Inve...,無
2,2241,艾姆勒,2022-01-26,14:30,線上法人說明會,本公司110年第4季營運狀況及財務資訊,http://www.amulaire.com/invest/stock,,無
3,8046,南電,2022-01-26,14:30,線上法說會,本公司受邀參加玉山證券舉辦之線上法說會，說明本公司之營運概況,http://www.nanyapcb.com.tw/nypcb/chinese/Inves...,,無
4,2303,聯電,2022-01-25,17:00,線上法說會,本公司2021年第四季財務暨營運報告。,http://www.umc.com,\n影音資訊網址：\nhttps://www.umc.com/zh-TW/Download/...,無
...,...,...,...,...,...,...,...,...,...
61,3711,日月光投控,2022-01-05,10:00,電話會議,無。,https://ir.aseglobal.com/c/index.php,,簡報檔內容與110年10月28日召開之法人說明會相同。
62,3596,智易,2022-01-05,09:00,電話會議,本公司受邀參加由Credit Suisse主辦之線上法說會「2022 Greater Chi...,https://www.arcadyan.com,,無
63,3545,敦泰,2022-01-04,09:45,線上法說會,受邀參加元富證券與證交所合辦之投資論壇「智慧台灣2022」,無,,無
64,6271,同欣電,2022-01-04,09:45,線上舉辦,受邀參加元富證券與證交所合辦之投資論壇「智慧台灣2022」,無,,無


In [13]:
df.to_csv("twseTest.csv", index = False)